importing the library for the API and setting the basin of interest (N Atlantic - includes the gulf coast area)

setting the list of cities we are interested in

In [35]:
from tropycal import tracks
import os
import numpy as np
import csv
from datetime import datetime
import math
#set area of interest
basin = tracks.TrackDataset(basin='north_atlantic',include_btk=False)

# the gulf coast cities we were given
target_cities = {
    "New Orleans, USA": (29.9511, -90.0715),
    "Houston, USA": (29.7604, -95.3698),
    "Tampa, USA": (27.9506, -82.4572),
    "Miami, USA": (25.7617, -80.1918),
    "Corpus Christi, USA": (27.8006, -97.3964),
    "Pensacola, USA": (30.4213, -87.2169),
    "Mobile, USA": (30.6954, -88.0399),
    "Galveston, USA": (29.3013, -94.7977),
    "Biloxi, USA": (30.3960, -88.8853),
    "Key West, USA": (24.5551, -81.7800),
    "Veracruz, Mexico": (19.1738, -96.1342),
    "Tampico, Mexico": (22.2553, -97.8686),
    "Campeche, Mexico": (19.8453, -90.5235),
    "Cancún, Mexico": (21.1619, -86.8515),
    "Mérida, Mexico": (20.9674, -89.5926),
    "Ciudad del Carmen, Mexico": (18.6491, -91.8071),
    "Progreso, Mexico": (21.2836, -89.6645),
    "Coatzacoalcos, Mexico": (18.1489, -94.4202),
    "Tuxpan, Mexico": (20.9589, -97.4044),
    "Havana, Cuba": (23.1136, -82.3666),
    "Varadero, Cuba": (23.1547, -81.2546),
    "Cienfuegos, Cuba": (22.1613, -80.4490),
    "Belize City, Belize": (17.5046, -88.1962),
    "George Town, Cayman Islands": (19.2869, -81.3674),
    "Nassau, Bahamas": (25.0343, -77.3963)
}

just_coords = [
    (29.9511, -90.0715),
    (29.7604, -95.3698),
    (27.9506, -82.4572),
    (25.7617, -80.1918),
    (27.8006, -97.3964),
    (30.4213, -87.2169),
    (30.6954, -88.0399),
    (29.3013, -94.7977),
    (30.3960, -88.8853),
    (24.5551, -81.7800),
    (19.1738, -96.1342),
    (22.2553, -97.8686),
    (19.8453, -90.5235),
    (21.1619, -86.8515),
    (20.9674, -89.5926),
    (18.6491, -91.8071),
    (21.2836, -89.6645),
    (18.1489, -94.4202),
    (20.9589, -97.4044),
    (23.1136, -82.3666),
    (23.1547, -81.2546),
    (22.1613, -80.4490),
    (17.5046, -88.1962),
    (19.2869, -81.3674),
    (25.0343, -77.3963)
]

currentPoint = target_cities["New Orleans, USA"]

basin.analogs_from_point((currentPoint[0], currentPoint[1]),radius=50)

--> Starting to read in HURDAT2 data
--> Completed reading in HURDAT2 data (1.48 seconds)
--> Starting to interpolate storms
--> Completed interpolating storms (5.8 seconds)


{'AL011860': np.float64(38.3),
 'AL051869': np.float64(32.9),
 'AL131887': np.float64(42.6),
 'AL041892': np.float64(17.5),
 'AL081893': np.float64(29.9),
 'AL011914': np.float64(21.0),
 'AL061915': np.float64(14.7),
 'AL041936': np.float64(12.4),
 'AL091936': np.float64(39.4),
 'AL041939': np.float64(39.6),
 'AL041947': np.float64(11.6),
 'AL051948': np.float64(8.9),
 'AL011955': np.float64(32.8),
 'AL051955': np.float64(22.2),
 'AL051971': np.float64(9.6),
 'AL111971': np.float64(15.2),
 'AL061975': np.float64(24.7),
 'AL181975': np.float64(8.8),
 'AL151977': np.float64(42.7),
 'AL041979': np.float64(33.5),
 'AL021988': np.float64(3.6),
 'AL071988': np.float64(9.6),
 'AL012001': np.float64(30.4),
 'AL022002': np.float64(19.8),
 'AL102002': np.float64(14.9),
 'AL032003': np.float64(43.7),
 'AL032005': np.float64(28.7),
 'AL122005': np.float64(45.9),
 'AL032020': np.float64(5.5),
 'AL282020': np.float64(10.6),
 'AL032021': np.float64(29.0)}

# A: Visualize storm tracks over the last 25-year period for the Gulf Coast region.

### For each city of interest, pull all storms of at least Cat1 that pass within the set radius (currently 50km) within the defined 30yr period

### Print the IDs for each storm and plot their courses and dump those plots into the specified folder

In [ ]:
# Define the folder path
output_folder = "/Users/andrew/COSC-3337/hurricanePlots"  # Change this to your desired folder path
# output_folder = "C:\Users\Andrew\Documents\COSC-3337\hurricanePlots"
os.makedirs(output_folder, exist_ok=True)

yearRange = (1998,2023)
radius = 60 #km
# stormLog = {}

# plot each city and its storms on a separate map (to make easier to read)
for city,coords in target_cities.items():
    print("{} is at {}".format(city, coords))

    # Set the full path for the plot image file
    savePathCity = city.split(",")
    save_path = os.path.join(output_folder, savePathCity[0]+".png")

    storms = basin.analogs_from_point(coords,radius=radius,year_range=yearRange,thresh={'v_min':33})
    
    if not storms:
        print("No hurricanes hit within 50km of {} between 1990 and 2020\n".format(savePathCity[0]))
    else:
        print("Storm log for {}: {}\n".format(savePathCity[0], storms))
        #thresh is min sustained wind of 33kt => 62 kmph
        basin.plot_analogs_from_point(coords,radius=radius,domain='north_atlantic', prop={'plot_names':True},year_range=yearRange,thresh={'v_min':33},save_path=save_path)



#plot all hurricanes (at least tropical storm) that hit any of the cities we care about in the last 25 years
#thresh is min sustained wind of 33kt => 62 kmph

#This is the list of storms that meet the criteria we're going to be plotting in the next line
storms = basin.analogs_from_shape(just_coords,year_range=yearRange,thresh={'v_min':33})
print(storms)
print(len(storms))

#domain is used to set the focus of the plot. would probably help to set a custom domain since none of the presets are the gulf coast area

save_path = os.path.join(output_folder, "gulf_coast_region.png")
basin.plot_storms(storms,domain='north_atlantic', prop={'plot_names':True},save_path=save_path) #dots

# basin.plot_storms(storms[:10],domain='north_atlantic', prop={'dots':False,'linecolor':'category','linewidth':2,'plot_names':True},save_path=save_path) #lines

    

In [38]:
def haversine_distance(coord1, coord2):
    # Calculate the great-circle distance between two points using the Haversine formula
    R = 6371  # Radius of Earth in kilometers
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    
    a = math.sin(dlat / 2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    return R * c

def find_closest_coords(data_coords, just_coords):
    # Initialize variables to store the closest coordinate pair and the smallest distance found
    closest_data_coord = None
    closest_just_coord = None
    min_distance = float('inf')
    
    # Loop through each coordinate in the dictionary's coordinates
    for data_coord in data_coords:
        # Compare it with each coordinate in just_coords
        for ref_coord in just_coords:
            # Calculate the distance between the two coordinates
            distance = haversine_distance(data_coord, ref_coord)
            
            # Update the closest coordinate if a shorter distance is found
            if distance < min_distance:
                min_distance = distance
                closest_data_coord = data_coord
                closest_just_coord = ref_coord
    
    return closest_data_coord, closest_just_coord, min_distance

def find_closest_city(closest_just_coord, target_cities):
    closest_city = None
    min_distance = float('inf')
    
    for city, coord in target_cities.items():
        # Calculate distance to each city's coordinates
        distance = haversine_distance(closest_just_coord, coord)
        
        # Update the closest city if a shorter distance is found
        if distance < min_distance:
            min_distance = distance
            closest_city = city
    
    return closest_city

def append_hurricane_summary_to_csv(data, filename='hurricane_summary.csv'):
    # Define the critical fields to include in the CSV
    critical_fields = ['id', 'name', 'year', 'lat', 'lon', 'vmax', 'mslp']
    
    # Extract values for each critical field except time
    data_to_write = {field: data[field] for field in critical_fields if field in data}
    
    # Get start_time and end_time from the time list
    start_time = data['time'][0].isoformat() if isinstance(data['time'][0], datetime) else data['time'][0]
    end_time = data['time'][-1].isoformat() if isinstance(data['time'][-1], datetime) else data['time'][-1]

    # Find the closest coordinate in the data to any coordinate in just_coords
    data_coords = list(zip(data['lat'], data['lon']))
    closest_data_coord, closest_just_coord, min_distance = find_closest_coords(data_coords, just_coords)
    affected_city = find_closest_city(closest_just_coord, target_cities)
    
    # Check if file exists to decide if headers are needed
    file_exists = os.path.isfile(filename)
    
    # Write data to CSV
    with open(filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        
        # If the file is new, add headers
        if not file_exists:
            headers = ['id', 'name', 'year', 'start_time', 'end_time', 'init_lat', 'init_lon', 'vmax', 'mslp','impact_lat','impact_lon','city_impacted']
            writer.writerow(headers)
        
        # Prepare a row with the required data
        row = [
            data['id'],
            data['name'],
            data['year'],
            start_time,
            end_time,
            data['lat'][0],   # Using the initial latitude
            data['lon'][0],   # Using the initial longitude
            max(data['vmax']), # Maximum vmax during the period
            min(data['mslp']),  # Minimum mslp during the period
            closest_data_coord[0],
            closest_data_coord[1],
            affected_city
        ]
        
        writer.writerow(row)

storms = basin.analogs_from_shape(just_coords,year_range=yearRange,thresh={'v_min':33})


logOfStormDicts = []
#clear csv file before writing everything to it
with open('hurricane_summary.csv', 'w') as file:
    pass

#write each storm to the csv file, writes the headers the first time and just appends every time after
for eachStorm in storms:
    stormOfInterest = basin.get_storm(eachStorm)
    # print(stormOfInterest.to_dict())
    logOfStormDicts.append(stormOfInterest.to_dict()) #prob dont need
    
    append_hurricane_summary_to_csv(stormOfInterest.to_dict())



# B: Identify Common Patterns and Trends

# C: Make a report after performing statistical analysis of track frequency, intensity, motion vectors and duration.